In [1]:
from database.market import Market
from database.sec import SEC
from database.adatabase import ADatabase
import pandas as pd
import matplotlib.pyplot as plt
from processor.processor import Processor as p
from statistics import variance
from tqdm import tqdm
from modeler.modeler import Modeler as m
from datetime import datetime, timedelta

In [2]:
market = Market()

In [3]:
market.connect()
prices = market.retrieve("prices")
sp5 = market.retrieve("sp500")
corr = market.retrieve("stock_crypto_corr")
market.disconnect()

In [34]:
len(corr[corr["value"]<-0.5]["security"].unique())

519

In [4]:
main_db = ADatabase("strat_eg")

In [5]:
datasets = {}
for strat in ["econ","financial","speculation"]:
    strat_db = ADatabase(f"strategy_{strat}")
    strat_db.connect()
    sim = strat_db.retrieve("sim")
    strat_db.disconnect()
    strat_factors = [x for x in sim.columns if "prediction" in x or "score" in x]
    for factor in strat_factors:
        sim.rename(columns={factor:factor+f"_{strat}"},inplace=True)
    datasets[strat] = sim

In [6]:
financial = datasets["financial"]
econ = datasets["econ"][[x for x in datasets["econ"].columns if "econ" in x or x in ["year","quarter"]]]
speculation = datasets["speculation"]

In [7]:
quarterly_set = financial.merge(econ,on=["year","quarter"],how="left").groupby(["year","quarter","ticker"]).mean().reset_index()

In [8]:
prices = p.column_date_processing(prices)
prices["year"] = [x.year for x in prices["date"]]
prices["week"] = [x.week for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]

In [9]:
weekly_set = prices.merge(speculation,on=["year","week","ticker"],how="left").dropna()

In [12]:
data = weekly_set.merge(quarterly_set,on=["year","quarter","ticker"],how="left")

In [17]:
for strat in ["econ","financial","speculation"]:
    data[f"{strat}_prediction"] = (data[f"xgb_prediction_{strat}"] + data[f"skl_prediction_{strat}"]  + data[f"cat_prediction_{strat}"])/3
    data[f"{strat}_delta"] = (data[f"{strat}_prediction"] - data["adjclose"])/data["adjclose"]

In [25]:
simulation = data[["date","ticker","adjclose","financial_delta","speculation_delta"]].dropna()

In [50]:
simulation[simulation["date"]>datetime(2020,3,1)]

,date,ticker,adjclose,financial_delta,speculation_delta
4319,2020-03-02,MMM,144.241754,0.158485,0.024540
4320,2020-03-03,MMM,136.908067,0.220541,0.079421
4321,2020-03-04,MMM,145.146682,0.151262,0.018152
4322,2020-03-05,MMM,142.724115,0.170803,0.035434
4323,2020-03-06,MMM,144.835613,0.153735,0.020339
...,...,...,...,...,...
2012758,2021-12-30,WMT,143.170000,-0.070143,-0.013879
2012759,2021-12-31,WMT,144.690000,-0.079911,-0.024238
2012760,2021-12-29,WBA,52.250000,0.602189,0.000968
2012761,2021-12-30,WBA,51.990000,0.610201,0.005973


In [26]:
industries = list(sp5["GICS Sector"].unique())

In [62]:
prediction_horizons = {"financial":90,"speculation":14}
trades = []
positions = 5
start_date = datetime(2020,1,1)
end_date = datetime(2021,12,15)
main_db.connect()
position_list = []
for strat in ["financial","speculation"]:
    prediction_horizon = prediction_horizons[strat]
    limit = prediction_horizon * 5
    for position in range(positions):
        date = start_date
        iterration_sim = simulation.copy()
        current_positions = pd.DataFrame(position_list)
        while date < end_date:
            try:
                if len(position_list) > 0 and position > 0:
                    current_tickers = current_positions[(current_positions["start_date"]>=date) & (current_positions["end_date"]<=date)]["ticker"]      
                    if len(current_tickers) > 0:
                        allowed_tickers = corr[(corr["index"].isin(current_tickers)) & (corr["value"]<=0)]["security"].unique()
                    else:
                        allowed_tickers = sp5["Symbol"].unique()
                else:
                    allowed_tickers = sp5["Symbol"].unique()
                todays_recs = iterration_sim[(iterration_sim["date"]==date) & (iterration_sim["ticker"].isin(allowed_tickers))]
                todays_recs.sort_values(f"{strat}_delta",ascending=False,inplace=True)
                if todays_recs.index.size > 0:
                    offering = todays_recs.iloc[position]
                    if offering[f"{strat}_delta"] > 0:
                        trade = offering
                        req = offering[f"{strat}_delta"]
                        ticker = trade["ticker"]
                        buy_price = trade["adjclose"]
                        due_date = int(min(limit,(simulation["date"].max()-date).days))
                        exits = iterration_sim[(iterration_sim["ticker"]==ticker) & (iterration_sim["date"]>date)].iloc[:due_date]
                        exits["gains"] = (exits["adjclose"] - buy_price) / buy_price
                        gain_exits = exits[exits["gains"]>=req].sort_values("date")
                        if gain_exits.index.size < 1:
                            exit = iterration_sim[(iterration_sim["ticker"]==ticker) & (iterration_sim["date"]>date)].iloc[due_date]
                            trade["sell_price"] = exit["adjclose"]
                        else:
                            exit = gain_exits.iloc[0]
                            trade["sell_price"] = buy_price * (1+(req))
                        trade["sell_date"] = exit["date"]
                        position_list.append({"ticker":ticker,"end_date":exit["date"],"start_date":date})
                        date = exit["date"] + timedelta(days=1)
                        trade["position"] = position
                        trade["strat"] = strat
                        main_db.store("trades_siloed_market_corr",pd.DataFrame([trade.to_dict()]))
                    else:
                        date = date + timedelta(days=1)
                else:
                    date = date + timedelta(days=1)
            except Exception as e:
                print(date,str(e))
                date = date+timedelta(days=1)
    main_db.disconnect()

2021-10-18 00:00:00 single positional indexer is out-of-bounds
2021-10-19 00:00:00 single positional indexer is out-of-bounds
2021-10-20 00:00:00 single positional indexer is out-of-bounds
2021-10-21 00:00:00 single positional indexer is out-of-bounds
2021-10-22 00:00:00 single positional indexer is out-of-bounds
2021-10-25 00:00:00 single positional indexer is out-of-bounds
2021-10-26 00:00:00 single positional indexer is out-of-bounds
2021-10-27 00:00:00 single positional indexer is out-of-bounds
2021-10-28 00:00:00 single positional indexer is out-of-bounds
2021-10-29 00:00:00 single positional indexer is out-of-bounds
2021-11-01 00:00:00 single positional indexer is out-of-bounds
2021-11-02 00:00:00 single positional indexer is out-of-bounds
2021-11-03 00:00:00 single positional indexer is out-of-bounds
2021-11-04 00:00:00 single positional indexer is out-of-bounds
2021-11-05 00:00:00 single positional indexer is out-of-bounds
2021-11-08 00:00:00 single positional indexer is out-of

2021-11-08 00:00:00 single positional indexer is out-of-bounds
2021-11-09 00:00:00 single positional indexer is out-of-bounds
2021-11-10 00:00:00 single positional indexer is out-of-bounds
2021-11-11 00:00:00 single positional indexer is out-of-bounds
2021-11-12 00:00:00 single positional indexer is out-of-bounds
2021-11-15 00:00:00 single positional indexer is out-of-bounds
2021-11-16 00:00:00 single positional indexer is out-of-bounds
2021-11-17 00:00:00 single positional indexer is out-of-bounds
2021-11-18 00:00:00 single positional indexer is out-of-bounds
2021-11-19 00:00:00 single positional indexer is out-of-bounds
2021-11-22 00:00:00 single positional indexer is out-of-bounds
2021-11-23 00:00:00 single positional indexer is out-of-bounds
2021-11-24 00:00:00 single positional indexer is out-of-bounds
2021-11-26 00:00:00 single positional indexer is out-of-bounds
2021-11-29 00:00:00 single positional indexer is out-of-bounds
2021-11-30 00:00:00 single positional indexer is out-of

2021-11-03 00:00:00 single positional indexer is out-of-bounds
2021-11-04 00:00:00 single positional indexer is out-of-bounds
2021-11-05 00:00:00 single positional indexer is out-of-bounds
2021-11-08 00:00:00 single positional indexer is out-of-bounds
2021-11-09 00:00:00 single positional indexer is out-of-bounds
2021-11-10 00:00:00 single positional indexer is out-of-bounds
2021-11-11 00:00:00 single positional indexer is out-of-bounds
2021-11-12 00:00:00 single positional indexer is out-of-bounds
2021-11-15 00:00:00 single positional indexer is out-of-bounds
2021-11-16 00:00:00 single positional indexer is out-of-bounds
2021-11-17 00:00:00 single positional indexer is out-of-bounds
2021-11-18 00:00:00 single positional indexer is out-of-bounds
2021-11-19 00:00:00 single positional indexer is out-of-bounds
2021-11-22 00:00:00 single positional indexer is out-of-bounds
2021-11-23 00:00:00 single positional indexer is out-of-bounds
2021-11-24 00:00:00 single positional indexer is out-of

In [59]:
if len(position_list) > 0 and position > 0:
    current_tickers = current_positions[(current_positions["start_date"]>=date) & (current_positions["end_date"]<=date)]["ticker"]      
    if len(current_tickers) > 0:
        allowed_tickers = corr[(corr["index"].isin(current_tickers)) & (corr["value"]<=-0.5)]["security"].unique()
    else:
        allowed_tickers = sp5["Symbol"].unique()
else:
    allowed_tickers = sp5["Symbol"].unique()